# K-Anonimato

In [1]:
import random as rdm
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("salario.csv")
df.head()

,id,pais,estado,cidade,genero,data,salario
0,1,United States,Alabama,Mobile,Male,10/01/1987,$5407.86
1,2,Portugal,Porto,Rego,Male,29/11/2007,$7833.82
2,3,Sweden,Kalmar,Emmaboda,Male,23/06/1984,$5530.85
3,4,Sweden,Västmanland,Sala,Female,04/11/1983,$2803.76
4,5,France,Bretagne,Quimperlé,Female,18/11/1990,$2201.87


In [3]:
df.describe()

,id
count,1000.000000
mean,500.500000
std,288.819436
min,1.000000
25%,250.750000
50%,500.500000
75%,750.250000
max,1000.000000


## Joining the locality attributes

In [4]:
ids = df.id
cities = df.cidade
states = df.estado
countries = df.pais

In [5]:
locality = []
for i in range(len(cities)):
    locality.append([ids[i], str('{}, {}, {}'.format(cities[i], states[i], countries[i]))])

In [6]:
df_locality = pd.DataFrame(locality, columns=['id', 'localidade'])
df_locality.head()

,id,localidade
0,1,"Mobile, Alabama, United States"
1,2,"Rego, Porto, Portugal"
2,3,"Emmaboda, Kalmar, Sweden"
3,4,"Sala, Västmanland, Sweden"
4,5,"Quimperlé, Bretagne, France"


## Joining the dataframes

In [7]:
join_dataset = pd.concat([df, df_locality], axis=1, join='inner')
join_dataset.head()

,id,pais,estado,cidade,genero,data,salario,id,localidade
0,1,United States,Alabama,Mobile,Male,10/01/1987,$5407.86,1,"Mobile, Alabama, United States"
1,2,Portugal,Porto,Rego,Male,29/11/2007,$7833.82,2,"Rego, Porto, Portugal"
2,3,Sweden,Kalmar,Emmaboda,Male,23/06/1984,$5530.85,3,"Emmaboda, Kalmar, Sweden"
3,4,Sweden,Västmanland,Sala,Female,04/11/1983,$2803.76,4,"Sala, Västmanland, Sweden"
4,5,France,Bretagne,Quimperlé,Female,18/11/1990,$2201.87,5,"Quimperlé, Bretagne, France"


## Removing attributes

In [8]:
dataset = join_dataset.drop(columns = ['id', 'pais', 'estado', 'cidade'])
dataset.describe()

,genero,data,salario,localidade
count,1000,1000,1000,1000
unique,2,956,1000,769
top,Female,17/01/1981,$3615.07,"Stockholm, Stockholm, Sweden"
freq,542,3,1,15


## Date generalization function

In [9]:
def date_generalization(dataset, level):
    if level == 1:
        # date '**/MM/YYYY'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**' + dataset.iloc[data]['data'][2:10])
    elif level == 2:
        # date '**/**/YYYY'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**/**' + dataset.iloc[data]['data'][5:10])
    elif level == 3:
        # date'**/**/****'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**/**/****')
        
    return dataset

In [10]:
def locality_generalization(dataset, level):
    if level == 0:
        # locality 'city'
        for data in range(len(dataset)):
            place = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = place[0] + ","+place[1] + "," + place[2]
    elif level == 1:
        # locality 'state,'
        for data in range(len(dataset)):
            place = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = "**," + place[1] + "," + place[2]
    elif level == 2:
        # locality 'country'
        for data in range(len(dataset)):
            place = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = "**,**," + place[2]
    elif level == 3:
        # locality '*'
        for data in range(len(dataset)):
            dataset.iloc[data]['localidade'] = '*'
        
    return dataset

In [11]:
def gender_generalization(dataset, level):
    if level == 1:
        # gender '*'
        for data in range(len(dataset)):
            dataset.iloc[data]['genero'] = '*'
            
    return dataset

## Precision metric

$precision (D, Na, h, HGVA) = 1 - \frac{\sum_{i=1}^{Na} \sum_{j=1}^{|D|} \frac{h}{HGVA_i}}{|D| * Na}
$

- D: dataset
- Na: number of semi-identifier attributes
- h: height of attribute value generalization hierarchy Ai after generalization
- |HGVAi|: maximum height of hierarchy

In [12]:
def precision(D, Na, HGVA):
    h = []
    summation = 0
    for j in range(len(D)):
        data = D.iloc[j]["data"].count('*')
        localidade = D.iloc[j]["localidade"].count('*')
        genero = D.iloc[j]["genero"].count('*')
        
        h_data = 0
        if data == 2:
            h_data = 1
        elif data == 4:
            h_data = 2
        elif data == 8:
            h_data = 3
            
        h_localidade = 0
        if localidade == 1:
            h_localidade = 3
        elif localidade == 2:
            h_localidade = 1
        elif localidade == 4:
            h_localidade = 2
            
        h_genero = 0
        if genero == 1:
            h_genero = 1
            
        h.append([h_data, h_genero, h_localidade])
    
    for i in range(Na):
        for j in range(len(D)):
            summation += (h[j][i] / (HGVA[i]))
    
    return 1 - (summation / (len(D) * Na))

In [13]:
def final_iterate(x,y):
    print()

# Generating anonymization

In [14]:
def anonymize(dataset, k):

    d = {'data': [], 'genero': [], 'localidade':[], 'salario':[]}
    df_final = pd.DataFrame(data=d)

    Ordem_Generalizacoes = [(date_generalization, 1),
                            (locality_generalization, 1),
                            (date_generalization, 2),
                            (locality_generalization, 2),
                            (gender_generalization, 1),
                            (date_generalization, 3),
                            (locality_generalization, 3),
                            (final_iterate, 1)]
        
    for generalizacao_atual in Ordem_Generalizacoes:
        aux = dataset.groupby(["data","genero","localidade"])["localidade"].count()

        atributos_K_anonimizados = aux[aux >= k].index

        for i in atributos_K_anonimizados:
            date = i[0]
            gender = i[1]
            locality = i[2]

            indices_prontos = dataset[ (dataset['data'] == date) & (dataset['genero'] == gender) & (dataset["localidade"] == locality)].index#[0]

            df_final = df_final.append(dataset.iloc[indices_prontos], ignore_index = True)
            dataset.drop(indices_prontos, inplace=True)
            dataset = dataset.reset_index(drop=True)
        
        generalizacao_atual[0](dataset, generalizacao_atual[1])
        
    calculate_precision = float("{0:.4f}".format(precision(df_final, 3, [4, 1, 3])))

    print('Precision for k =', k, 'is', calculate_precision)
    df_final.to_csv (r'output/export_dataframe_k_' + str(k) +'.csv', index = None, header=True)
    
    dataset_anonymized = open('output/export_dataframe_k_'  + str(k) +'.csv',"a")
    dataset_anonymized.write('Precision is ' + str(calculate_precision))
    dataset_anonymized.close()

In [15]:
for k in [2, 4, 8, 16]:
    anonymize(dataset.copy(), k)

C:\Users\Tibet\AppData\Local\conda\conda\envs\jup\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)



Precision for k = 2 is 0.6001

Precision for k = 4 is 0.4329

Precision for k = 8 is 0.2544

Precision for k = 16 is 0.1944
